# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4873, uptime 0:00:33
mini_beamline                    RUNNING   pid 4874, uptime 0:00:33
random_walk                      RUNNING   pid 4875, uptime 0:00:33
random_walk_horiz                RUNNING   pid 4876, uptime 0:00:33
random_walk_vert                 RUNNING   pid 4877, uptime 0:00:33
simple                           RUNNING   pid 4878, uptime 0:00:33
thermo_sim                       RUNNING   pid 4879, uptime 0:00:33
trigger_with_pc                  RUNNING   pid 4880, uptime 0:00:33


In [2]:
%run scripts/beamline_configuration.py

/home/travis/virtualenv/python3.7.1/lib/python3.7/site-packages/pims/image_reader.py:26: RuntimeWarning: PIMS image_reader.py could not find scikit-image. Falling back to matplotlib's imread(), which uses floats instead of integers. This may break your scripts. 
(To ignore this warning, include the line "warnings.simplefilter("ignore", RuntimeWarning)" in your script.)
  warnings.warn(RuntimeWarning(ski_preferred))


## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 3     Time: 2020-07-02 20:47:46
Persistent Unique Scan ID: 'c72ce5bb-b183-4f6f-a96a-4357085f48f2'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 20:47:46.1 |     -1.000 |     100453 |
|         2 | 20:47:46.2 |     -0.500 |     100492 |


|         3 | 20:47:46.2 |      0.000 |     101161 |
|         4 | 20:47:46.3 |      0.500 |      99879 |


|         5 | 20:47:46.3 |      1.000 |      98363 |
+-----------+------------+------------+------------+
generator scan ['c72ce5bb'] (scan num: 3)





('c72ce5bb-b183-4f6f-a96a-4357085f48f2',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/c72ce5bb-b183-4f6f-a96a-4357085f48f2-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 4     Time: 2020-07-02 20:47:46
Persistent Unique Scan ID: '800a6fda-1429-47e1-8eec-24ae8c0dbab1'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 20:47:46.7 |          -1 | 1076891281 |


|         2 | 20:47:46.9 |          -0 | 1066437471 |


|         3 | 20:47:47.0 |           0 | 1061652132 |


|         4 | 20:47:47.1 |           0 | 1057495541 |


|         5 | 20:47:47.2 |           1 | 1055087190 |
+-----------+------------+-------------+------------+
generator scan ['800a6fda'] (scan num: 4)





('800a6fda-1429-47e1-8eec-24ae8c0dbab1',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 5     Time: 2020-07-02 20:47:47
Persistent Unique Scan ID: 'bebeacbc-3589-4024-be74-39718f689c64'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 20:47:47.7 | 1055440843 |


|         2 | 20:47:48.6 | 1105861787 |


|         3 | 20:47:49.6 | 1129318730 |


...

|        28 | 20:48:14.6 | 1075844248 |


|        29 | 20:48:15.6 | 1057096852 |


|        30 | 20:48:16.6 | 1109693771 |


+-----------+------------+------------+
generator count ['bebeacbc'] (scan num: 5)





('bebeacbc-3589-4024-be74-39718f689c64',)